In [ ]:
import os
import sys
import numpy as np
import numpy.random as nrand
import pandas as pd
import pickle
import copy
import matplotlib.pyplot as plt
from itertools import combinations
from matplotlib.ticker import FormatStrFormatter
import scipy
import scipy.sparse.linalg
from scipy.linalg import hadamard
from scipy.optimize import curve_fit
import itertools
from sklearn.linear_model import LinearRegression
currentdir = os.path.abspath('')
targetdir = os.path.dirname(currentdir)+'/utils'
sys.path.append(targetdir)
from utils import *

In [ ]:
def get_distribution(landscape,measurement_std,rep):
    fitness_rep = []
    for _ in range(rep):
        fitness_rep.append(Add_Error(landscape,measurement_std))
    fitness_rep = np.array(fitness_rep)

    duplicates_list = list(range(rep))
    ruggedness_dict = {i:[] for i in range(1,rep+1)}
    tmplate = copy.deepcopy(landscape)
    for duplication in range(1,rep+1):
        if duplication == 1:
            iter_list = duplicates_list
            for replicate in iter_list:
                ruggedness_dict[duplication].append(get_ruggedness(fitness_rep[replicate]))
        else:
            iter_list = combinations(duplicates_list,duplication)   
            for replicate in iter_list:
                tmplate[:,N] = np.average(fitness_rep[list(replicate),:,N],axis=0)
                ruggedness_dict[duplication].append(get_ruggedness(tmplate))

    ground_truth = get_ruggedness(landscape)
    return ground_truth,ruggedness_dict

In [ ]:
model_list = ['Polynomial'] # Type of theoretical landscape model: 'NK', 'RMF', 'Polynomial'
N_list = [5,10,15] # Number of variable sites: 5, 10, 15
metric_list = ['N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs']
    # Type of metrics: 'N_max', 'epi', 'r_s', 'open_ratio','E', 'gamma', 'adptwalk_steps', 'adptwalk_probs'

for N in N_list:
    print(f"N:{N}")
    load_pregenerated_data(N)
    
    for model in model_list:
        print(f"  model:{model}")

        for metric in metric_list:
            print(f"    {metric}")
            with open(f'../FL_data_3X10/{model}{N}_{metric}_landscape_3X10.pkl', 'rb') as f:
                landscape_list = pickle.load(f)
            
            get_ruggedness = get_ruggedness_function(metric,N,gt_code=landscape_list[1][0][:,:N])
            
            res_dict = {i:[] for i in ['ground_truth','N_site','model','replication',
                           'metric','ruggedness','measurement_std','landscape_rep','error_rep','raw']}

            for ruggedness in range(1,4):
                for landscape_rep in range(3):
                    landscape = landscape_list[ruggedness][landscape_rep]
                    for measurement_std in [0.1/3*i for i in [1,2,3,4]]:
                        for error_rep in range(3):
                            for rep in [3,4,5,6]:
                                print(f'{ruggedness}:{landscape_rep}:{measurement_std}:{error_rep}:{error_rep}:{rep}')
                                ground_truth,raw = get_distribution(landscape,measurement_std,rep)
                                res_dict['ground_truth'].append(ground_truth)
                                res_dict['N_site'].append(N)
                                res_dict['model'].append(model)
                                res_dict['replication'].append(rep)
                                res_dict['metric'].append(metric)
                                res_dict['ruggedness'].append(ruggedness)
                                res_dict['measurement_std'].append(measurement_std)
                                res_dict['landscape_rep'].append(landscape_rep)
                                res_dict['error_rep'].append(error_rep)
                                res_dict['raw'].append(raw)
            
            with open(f'raw_data/{model}{N}_{metric}_raw.pkl', 'wb') as f:
                pickle.dump(res_dict,f)

            
            